### 노트북 설명

이 노트북은 사용자 커스텀 전략이 시스템에서 올바르게 작동하는지 검증하기 위한 파일입니다. Google Colab 환경에 맞게 설정되어 있으며, 로컬 환경에서 사용하는 경우 경로 설정에 유의해야 합니다.

**주요 단계:**

1.  **Google Drive 마운트:** Google Drive에 연결하여 필요한 파일에 접근합니다.
2.  **선물 전략 검증:**
    *   전략 파일과 설정 파일을 가져옵니다.(Path 유의)
    *   가져온 전략 및 설정 파일을 사용하여 시스템 데이터 모듈(가짜)을 기반으로 전략을 실행합니다.
    *   전략 실행 결과로 **`"symbol:weights"`** 딕셔너리가 출력되는지 확인합니다.
    *   가중치 절댓값의 합이 1이 넘는지 체크합니다. 1을 초과 할 경우, 전략을 다시 체크하십시오.

**참고:** 사용자 지정 전략을 테스트하려면 해당 전략 파일과 설정 파일의 가져오기 경로를 수정하고 노트북을 실행하십시오.

#### - Original GitHub Link : https://github.com/NeoMatrixAI/nb-runner/tree/main/notebooks

# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Verify Futures strategy

In [2]:
# Create Fake module
import sys
import types
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np

# 1. Create a fake module called 'module' and register it with the system.
module_obj = types.ModuleType('module')
sys.modules['module'] = module_obj

# 2. Create a fake empty module called 'module.data_context' and register it with the system.
data_context_module = types.ModuleType('module.data_context')
sys.modules['module.data_context'] = data_context_module

# 3. Define the DataContext class required by the strategy.
class DataContext(ABC):
    @property
    @abstractmethod
    def current_dt(self) -> datetime: pass
    @abstractmethod
    def get_history(self, assets: list, window: int, frequency: str, fields: str or list = 'close') -> pd.DataFrame: pass

# 4. Place the defined DataContext class inside the fake 'module.data_context' module.
data_context_module.DataContext = DataContext

print("✅ Fake 'module.data_context' module and DataContext class created successfully.")

✅ Fake 'module.data_context' module and DataContext class created successfully.


In [ ]:
import sys
import yaml

# ============================================================================
# 1. Import strategy and load config.yaml
# ============================================================================
# [TODO] Replace with your own strategy file path
from drive.MyDrive.NeoMatrixAI.futures.your_strategy.your_strategy import strategy

# [TODO] Replace with your own config.yaml path
config_path = "/content/drive/MyDrive/NeoMatrixAI/futures/your_strategy/config.yaml"
with open(config_path, 'r') as f:
    full_config = yaml.safe_load(f)

# Extract strategy section
strategy_section = full_config["strategy"]

# Build strategy_config based on your config.yaml structure
# Option 1: If your params are directly under 'strategy' (flat structure)
#   e.g., strategy.assets, strategy.window, strategy.bb_period, ...
# Option 2: If your params are nested under 'strategy.config' 
#   e.g., strategy.config.base, strategy.config.position, ...

if "config" in strategy_section:
    # Nested structure: merge assets/frequency with config contents
    strategy_config = {
        "assets": strategy_section["assets"],
        "frequency": strategy_section.get("frequency", "1m"),
        **strategy_section["config"]  # Unpack nested config (base, position, sltp, etc.)
    }
else:
    # Flat structure: use strategy section as-is
    strategy_config = strategy_section

print("strategy_config:", strategy_config)

# ============================================================================
# 2. Prepare mock objects and data required for testing.
# ============================================================================
class MockDataContext(DataContext):
    def __init__(self, fake_data):
        self._fake_data = fake_data

    @property
    def current_dt(self) -> datetime:
        return datetime.now()

    def get_history(self, assets, window, frequency, fields):
        return self._fake_data[fields]

def create_fake_data(assets, num_periods):
    """
    Create realistic fake OHLCV data for strategy testing.
    
    Features:
    - Random walk with mean reversion (realistic price movement)
    - Includes trends, pullbacks, and volatility clusters
    - RSI will vary naturally between 20-80
    - Prices will cross Bollinger Bands occasionally
    """
    end_time = pd.Timestamp.now(tz='UTC').floor('T')
    dates = pd.date_range(end=end_time, periods=num_periods, freq='1min')

    all_dfs = []
    for i, asset in enumerate(assets):
        base_price = 100 * (1 + i * 0.5)
        np.random.seed(42 + i)
        
        # Create realistic price movement with multiple regimes
        returns = np.zeros(num_periods)
        
        for j in range(num_periods):
            # Regime switching: trend, mean-reversion, high volatility
            regime = (j // 200) % 4
            
            if regime == 0:  # Uptrend
                returns[j] = np.random.normal(0.0003, 0.002)
            elif regime == 1:  # Downtrend
                returns[j] = np.random.normal(-0.0003, 0.002)
            elif regime == 2:  # High volatility
                returns[j] = np.random.normal(0, 0.005)
            else:  # Mean reversion / sideways
                returns[j] = np.random.normal(0, 0.001)
        
        prices = base_price * np.exp(np.cumsum(returns))

        # Add realistic OHLC relationships
        volatility = np.abs(returns) + 0.001
        df = pd.DataFrame({
            'datetime': dates,
            'open': prices * (1 + np.random.normal(0, 0.0005, num_periods)),
            'high': prices * (1 + volatility * np.random.uniform(0.5, 1.5, num_periods)),
            'low':  prices * (1 - volatility * np.random.uniform(0.5, 1.5, num_periods)),
            'close': prices,
            'volume': np.random.uniform(1000, 100000, num_periods) * (1 + np.abs(returns) * 100)
        })
        df['asset'] = asset
        all_dfs.append(df)

    df_concat = pd.concat(all_dfs, ignore_index=True)
    df_final = df_concat.set_index(['asset', 'datetime'])
    df_final = df_final.sort_values(by=['datetime', 'asset'])

    print(f"Generated {len(df_final)} rows of fake data for {len(assets)} assets")
    display(df_final)
    return df_final

# ============================================================================
# 3. Run verify test
# ============================================================================
# num_periods: adjust based on your strategy's window requirement
# e.g., if window=200, use at least 1440 (enough data for indicators to stabilize)
fake_data = create_fake_data(assets=strategy_config["assets"], num_periods=1440)
mock_context = MockDataContext(fake_data)

print("\n Strategy function testing...")
results = strategy(context=mock_context, config_dict=strategy_config)
print(" Strategy function execution complete!")

print("\n[Final weighted result]")
results

In [8]:
abs_sum = sum(abs(info["weight"]) for info in results.values())

if abs_sum > 1:
    print(f"⚠️ Warning: The sum of absolute weights exceeds 1! (Current: {abs_sum:.4f})")
else:
    print(f"✅ OK: Sum of absolute weights = {abs_sum:.4f}")

✅ OK: Sum of absolute weights = 1.0000
